In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from datetime import date
  


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
#Extracting the tickers of the BSE:
tickers = pd.read_html('https://ournifty.com/stock-list-in-nse-fo-futures-and-options.html#:~:text=NSE%20F%26O%20Stock%20List%3A%20%20%20%20SL,%20%201000%20%2052%20more%20rows%20')[0]
tickers=tickers.SYMBOL.tolist()

#Tickers we want to test on:
#tick_test=tickers[0:20]
tick_test=tickers
#Updating the tickers so that it can be recognized and pulled from Yahoo Finance
for i in range(0, len(tick_test),1):
    tick_test[i]= tick_test[i]+'.NS'

#Importing the data
start_date= '2021-03-01'
today = date.today()
today_date= today.strftime('%Y-%m-%d')

data = yf.download(tick_test,start_date,today_date)

In [ ]:
#Finding all the unique col names of a given stock

# data.columns
# a=list()
# for item in data.columns:
#    a.append(item[0])
# a=np.array(a)
# np.unique(a)

In [ ]:
#Extracting the Close Price
close_price= data.loc[:, 'Close']

#Extracting the daily Volume
volume= data.loc[:, 'Volume']

#Number of NA values per column
my_dict={}
for col in close_price.columns:
    temp=close_price[col].isna().sum()
    my_dict[col]=temp
#print(my_dict)

#Dropping NA values from Rows and Columns in DataFrame
close_price.dropna(axis=0, how='all', inplace=True)
close_price.dropna(axis=1, how='all', inplace=True)

In [ ]:
#Plotting some of the return time series

fig, ax = plt.subplots(5, 4, figsize=(25,20))
nrow=5
ncol=4
counter=0

for i in range(0,nrow,1):
    for j in range(0,ncol,1):
        col=close_price.columns[counter]
        returns= np.log(close_price[col]/close_price[col].shift(-1))
        ax[i][j].hist(returns)
        ax[i][j].set_title(col)
        counter=counter+1
        


#for i in range(0,len(close_price.columns),1):
#    plt.subplot(5, 4, i+1)
#    col=close_price.columns[i]
#    plt.plot(close_price[col])

In [ ]:
# Creating the Sharpe Ratio Dataframe


avg_retrn_daily=[]
avg_retrn_annual=[]
std_dev_annual=[]
std_dev_daily=[]
sr_daily=[]
sr_annual=[]
start_price=[]
end_price=[]

vol_df=pd.DataFrame()
for col in close_price.columns:
    returns= np.log(close_price[col]/close_price[col].shift(1))
    daily_avg= np.mean(returns)
    annual_avg= daily_avg*252
    
    daily_std = np.std(returns)
    annual_std= daily_std* (252**0.5)
    
    daily_sr= daily_avg/daily_std
    annual_sr= annual_avg/annual_std
    
    avg_retrn_daily.append(daily_avg*100)
    avg_retrn_annual.append(annual_avg*100)
    std_dev_annual.append(annual_std*100)
    std_dev_daily.append(daily_std*100)
    sr_daily.append(daily_sr)
    sr_annual.append(annual_sr)
    start_price.append(close_price[col][0])
    end_price.append(close_price[col][-1])
    
vol_df.index=close_price.columns
vol_df["Avg Daily Return %"]=avg_retrn_daily
vol_df["Avg Annual Return %"]=avg_retrn_annual
vol_df["Daily Volatility %"]=std_dev_daily
vol_df["Annual Volatility %"]=std_dev_annual
vol_df["Daily Sharpe Ratio"]=sr_daily
vol_df["Annual Sharpe Ratio"]=sr_annual
vol_df["Start Price"]=start_price
vol_df["End Price"]=end_price

In [ ]:
#display(vol_df)


#Looking at the top 20 Stocks in the BSE by Sharpe Ratio
vol_df_sort= vol_df.sort_values(by='Annual Sharpe Ratio', ascending= False)
vol_df_sort.head(20)